# Digital Equity Data & Jupyter Notebooks
Eleanor Tutt | tutte@carnegielibrary.org | Code for PGH Work Night | June 13, 2018

## Welcome!

This is the same Jupyter notebook I will talk through. You can also use it as a starting point to explore digital equity data during the second half of the work night.

I have been working with Jupyter notebooks on and off for several years, but I still consider myself a beginner and this talk is designed for beginners!

I'll leave lots of time for you to explore Jupyter notebooks, python, and digital equity data using this notebook as a starting point - you can slow down or speed up then, preferably in small groups with new friends. :)

## Some helpful resources

#### Jupyter Notebooks

* https://mybinder.org/

#### Python / Pandas

* http://pandas.pydata.org/pandas-docs/stable/10min.html
* https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe
* https://chrisalbon.com/#python

#### Charting



#### Folium

* https://python-visualization.github.io/folium/quickstart.html
* http://mattgoldwasser.com/posts/choroplot/
* http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb?flush_cache=true


#### Digital Equity Data

* https://data.wprdc.org/dataset/libraries
* https://data.wprdc.org/dataset/clp-public-wifi
* https://www.fcc.gov/reports-research/maps/
* https://data.wprdc.org/dataset/pittsburgh-internet-service-providers-by-block-june-2016

## The Jupyter toolbar

In [ ]:
# No code for this part - just talk!

## Setting up your notebook

In [ ]:
# if you want to use any python packages, add them at the beginning of your notebook
# packages allow you to take "shortcuts" instead of coding everything yourself
# like if you are making a pizza and you buy dough instead of making it totally from scratch

# pandas is a python package popular with data scientists
# is common to shorten pandas to pd when you import it
import pandas as pd

# folium is a python package that helps create interactive maps using leaflet
import folium

# later we are going to read in some geojson
import json

# and this will help us with some colors for a choropleth map
from branca.colormap import linear

## Importing data

In [ ]:
# pandas has a function called read_csv that can import csv data and store it as a dataframe
# a dataframe is a common pandas object that stores information in rows and columns
# you can think of it a like a spreadsheet, especially if you are using .csv data

# here we are creating a dataframe named libraries from a previously downloaded csv file
# original data from https://data.wprdc.org/dataset/libraries
# you could also link directly to the wprdc download url in your code:
# https://data.wprdc.org/datastore/dump/14babf3f-4932-4828-8b49-3c9a03bae6d0

libraries = pd.read_csv("librarylocations.csv")

# here is library wifi use data - brand new on the WPRDC!
# original data from https://data.wprdc.org/dataset/clp-public-wifi

wifi = pd.read_csv("clp-public-wifi.csv") 

# here we are creating a dataframe named fcc from previously downloaded data
# original data from 
# we added the dype parameter to make sure tract numbers are imported as text and not numbers

fcc = pd.read_csv("tract_map_dec_2016.csv", dtype={'tractcode': str})


## Looking at your data

In [ ]:
# adding .info() to the name of a dataframe gives us some information about the dataframe -
# how many rows of data we have, what the columns are named, and what the data types are

fcc.info()

In [ ]:
fcc.info()

In [ ]:
# you also might want to look at the first few rows of your dataframe

libraries.head(5)

In [ ]:
# or the last few rows

fcc.tail(3)

In [ ]:
fcc.describe()

## Summarizing your data

## Making a chart

In [ ]:
# there are many, many options for making charts!

## Making a map

In [ ]:
m = folium.Map(location=[40.442, -79.997],
                        zoom_start=13,
                        tiles="Stamen Toner")

library_layer = folium.FeatureGroup(name='CLP Libraries')

for index, row in libraries.iterrows():
    library_layer.add_child(folium.CircleMarker(location=(row["Lat"],
                                                    row["Lon"]),
                            radius=150,
                            popup=row["Name"],
                            color=False,
                            fill_opacity=1,
                            fill_color="#d4237d"))

m.add_child(library_layer)

folium.LayerControl().add_to(m)

m

## Making a more complicated map!

In [ ]:
# side note: folium has a "choropleth" shortcut I generally use
# but I just learned today they are thinking of depreciating it
# so I made this map in a slightly more complicated way

In [ ]:
# our fcc dataframe has data for the entire US
# but all those census tracts would make for a large file
# we will load a previously downloaded geojson file
# original data from https://data.wprdc.org/dataset/allegheny-county-census-tracts-2016
# a good place to explore geojson as a data structure is http://geojson.io/

allegheny_tracts = json.load(open("Allegheny_County_Census_Tracts_2016.geojson"))

In [ ]:
# create a colormap

colormap = linear.YlGn_09.scale(
    fcc.pcat_all.min(),
    fcc.pcat_all.max())

print(colormap(5.0))

colormap

In [ ]:
# and a way to connect tractcode/geoids to the relevant data

fcc_dict = fcc.set_index('tractcode')['pcat_all']
fcc_dict['42003468900']

In [ ]:
m2 = folium.Map(location=[40.442, -79.997],
                        zoom_start=13,
                        tiles="Stamen Toner")

fcc_layer = folium.GeoJson(
    allegheny_tracts,
    name='Residential Internet',
    style_function=lambda feature: {
        'fillColor': colormap(fcc_dict[feature['properties']['GEOID']]),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.8,
    }
)

m2.add_child(fcc_layer)
m2.add_child(library_layer)


folium.LayerControl().add_to(m2)


m2